In [ ]:
import pandas as pd
import os
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack


ModuleNotFoundError: No module named 'ace_tools'

In [41]:

# Caminho relativo ao notebook atual
file_path_itens = "../data/preprocessed/itens/dados_preprocessados.csv"

# Carregar o CSV para um DataFrame
df_itens = pd.read_csv(file_path_itens)

# Exibir as primeiras linhas para conferir os dados
print(df_itens.head())


                                   page  \
0  13db0ab1-eea2-4603-84c4-f40a876c7400   
1  92907b73-5cd3-4184-8d8c-e206aed2bf1c   
2  61e07f64-cddf-46f2-b50c-ea0a39c22050   
3  30e2e6c5-554a-48ed-a35f-6c6691c8ac9b   
4  9dff71eb-b681-40c7-ac8d-68017ac36675   

                                                 url  \
0  http://g1.globo.com/am/amazonas/noticia/2022/0...   
1  http://g1.globo.com/pa/santarem-regiao/noticia...   
2  http://g1.globo.com/mundo/noticia/2022/07/08/e...   
3  http://g1.globo.com/politica/noticia/2021/09/0...   
4  http://g1.globo.com/politica/noticia/2021/09/1...   

                      issued                   modified  \
0  2022-06-18 20:37:45+00:00  2023-04-15 00:02:08+00:00   
1  2019-06-20 17:19:52+00:00  2023-06-16 20:19:15+00:00   
2  2022-07-08 08:55:52+00:00  2023-04-15 04:25:39+00:00   
3  2021-09-09 19:06:46+00:00  2023-06-07 17:44:54+00:00   
4  2021-09-15 19:16:13+00:00  2023-06-07 17:43:39+00:00   

                                               ti

In [42]:
print("Número total de linhas no DataFrame:", len(df_itens))
print(df_itens.count())  # Conta os valores NÃO NULOS por coluna
print(df_itens.isnull().sum())  # Mostra quantos valores nulos há por coluna


Número total de linhas no DataFrame: 255603
page        255603
url         255603
issued      255603
modified    255603
title       255340
body        255603
caption     255520
dtype: int64
page          0
url           0
issued        0
modified      0
title       263
body          0
caption      83
dtype: int64


In [43]:
df_itens = df_itens.dropna()


In [44]:
print("Número total de linhas no DataFrame:", len(df_itens))
print(df_itens.count())  # Conta os valores NÃO NULOS por coluna
print(df_itens.isnull().sum())  # Mostra quantos valores nulos há por coluna


Número total de linhas no DataFrame: 255275
page        255275
url         255275
issued      255275
modified    255275
title       255275
body        255275
caption     255275
dtype: int64
page        0
url         0
issued      0
modified    0
title       0
body        0
caption     0
dtype: int64


In [ ]:
df_itens = df_itens.drop(columns=["modified"])


In [67]:
df_itens.head()

,page,url,issued,title,body,caption
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,caso bruno dom suspeito prisão temporário decr...,audiência custódio amazona decretar sábado pri...,lima escoltar agente polícia federal Fórum Jus...
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,linguajar santareno diferenciar cheio identida...,vista aéreo santarém ádrio produção paraense v...,expressão santareno significar significar cois...
2,61e07f64-cddf-46f2-b50c-ea0a39c22050,http://g1.globo.com/mundo/noticia/2022/07/08/e...,2022-07-08 08:55:52+00:00,premiê shinzo abe morrer balear Japão,vídeo mostrar assassino shinzo abe atirar cost...,ministro atingir caseira discursar campanha ci...
3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,http://g1.globo.com/politica/noticia/2021/09/0...,2021-09-09 19:06:46+00:00,relator stf fachin vota marco temporal demarca...,relator stf fachin vota marco temporal demarca...,ministro defender posse indígena diferente pos...
4,9dff71eb-b681-40c7-ac8d-68017ac36675,http://g1.globo.com/politica/noticia/2021/09/1...,2021-09-15 19:16:13+00:00,voto pedir vista suspender julgamento stf dema...,pedir vista análiser processo ministro moraes ...,reivindicar demarcação terro ocupar data const...


In [71]:
# Criar diretório se não existir
output_dir = "../data/tfidf/itens/"
os.makedirs(output_dir, exist_ok=True)

# Função para calcular TF-IDF e salvar em CSV
def calculate_tfidf_and_save(df, columns, output_prefix):
    for column in columns:
        if column not in df.columns:
            print(f"⚠️ Coluna {column} não encontrada no DataFrame. Pulando...")
            continue
        
        print(f"📌 Processando TF-IDF para coluna: {column}")

        # Criando o modelo TF-IDF (max_features pode ser ajustado conforme necessário)
        vectorizer = TfidfVectorizer(max_features=500)
        tfidf_matrix = vectorizer.fit_transform(df[column].fillna("").astype(str))
        
        # Convertendo matriz esparsa para DataFrame
        tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

        # Adicionando colunas principais
        tfidf_df.insert(0, "page", df["page"])
        tfidf_df.insert(1, "url", df["url"])
        tfidf_df.insert(2, "issued", df["issued"])
        
        # Definindo nome do arquivo
        filename = os.path.join(output_dir, f"{output_prefix}_{column}.csv")
        
        # Salvando o arquivo
        tfidf_df.to_csv(filename, index=False, encoding='utf-8')
        print(f"✅ Arquivo salvo: {filename}")

# Lista de colunas a processar
columns_to_process = ["title", "body", "caption"]

# Chamando a função
calculate_tfidf_and_save(df_itens, columns_to_process, "tfidf_result")


📌 Processando TF-IDF para coluna: title
✅ Arquivo salvo: ../data/tfidf/itens/tfidf_result_title.csv
📌 Processando TF-IDF para coluna: body
✅ Arquivo salvo: ../data/tfidf/itens/tfidf_result_body.csv
📌 Processando TF-IDF para coluna: caption
✅ Arquivo salvo: ../data/tfidf/itens/tfidf_result_caption.csv


Carregando dataset de vallidação

In [5]:
file_path_validacao = "../data/validacao/validacao.csv"

# Carregar o CSV removendo quebras de linha e espaços extras
df_validacao = pd.read_csv(file_path_validacao, sep=",", skipinitialspace=True, engine="python")



Selecionando a coluna UserId

In [ ]:
# Selecionar apenas a coluna userId e salvar em uma variável
user_ids_validacao = df_validacao["userId"]

# Exibir as primeiras linhas para conferir
print(user_ids_validacao.head())



0    e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...
1    d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...
2    755062dd39a48809880cf363b04268c3af2c003088cde0...
3    ec1639851d99586c7f4da928deb49187303aec6e3b8d66...
4    a120515626fe5d12b22b7d5a7c5008912cc69284aa26cc...
Name: userId, dtype: object


Carregando dataset de Treino

In [ ]:
# Caminho da pasta onde os arquivos CSV estão armazenados
folder_path_treino = "../data/files/treino/" 

# Encontrar todos os arquivos CSV na pasta
csv_files = glob.glob(os.path.join(folder_path_treino, "*.csv"))

# Lista para armazenar os DataFrames
dataframes = []

# Ler cada arquivo CSV e adicioná-lo à lista de DataFrames
for file in csv_files:
    df_treino = pd.read_csv(file)
    dataframes.append(df_treino)

# Concatenar todos os DataFrames em um único DataFrame
df_final_treino = pd.concat(dataframes, ignore_index=True)

# Exibir as primeiras linhas do DataFrame final
print(df_final_treino.head())


                                              userId    userType  historySize  \
0  f98d1132f60d46883ce49583257104d15ce723b3bbda21...  Non-Logged            3   
1  2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...  Non-Logged           60   
2  0adffd7450d3b9840d8c6215f0569ad942e782fb19b805...      Logged          107   
3  c1e8d644329a78ea1f994292db624c57980b2886cfbc2d...  Non-Logged           56   
4  e777d1f31d4d955b63d60acc13df336d3903f52ab8f8f4...  Non-Logged            4   

                                             history  \
0  c8aab885-433d-4e46-8066-479f40ba7fb2, 68d2039c...   
1  3325b5a1-979a-4cb3-82b6-63905c9edbe8, fe856057...   
2  04756569-593e-4133-a95a-83d35d43dbbd, 29b6b142...   
3  1f2b9c2f-a2d2-4192-b009-09065da8ec23, 04756569...   
4  bebdeb3e-1699-43e0-a1b8-989f5a6ab679, f4b484a7...   

                                    timestampHistory  \
0        1657146417045, 1657146605778, 1657146698738   
1  1656684240278, 1656761266729, 1656761528085, 1...   
2  16566

Filtrar os ids do dataset de validação no dataset de treino

In [ ]:
# Filtrar o dataset de treino para manter apenas os IDs que estão na validação

df_filtrado = df_final_treino[df_final_treino["userId"].isin(user_ids_validacao)]

# Exibir as primeiras linhas do novo dataset filtrado
print(df_filtrado.head())

                                              userId    userType  historySize  \
3  c1e8d644329a78ea1f994292db624c57980b2886cfbc2d...  Non-Logged           56   
4  e777d1f31d4d955b63d60acc13df336d3903f52ab8f8f4...  Non-Logged            4   
5  6aa5109374f6534b0400ccbbdb9c7b7d64c40a663fe51c...  Non-Logged            4   
7  c33bcbcaf32fe895fb0a854c94531120657ad5adabe3e1...  Non-Logged            3   
8  3c23b4ee53e663cb2d47eed9b5e7216cb7e6784e7f7722...  Non-Logged            4   

                                             history  \
3  1f2b9c2f-a2d2-4192-b009-09065da8ec23, 04756569...   
4  bebdeb3e-1699-43e0-a1b8-989f5a6ab679, f4b484a7...   
5  7fe849c0-4a55-429d-b480-11ee216909dd, 2de36771...   
7  0b8f3728-4ef7-40e7-85ac-367d8039bb6d, c8ade35b...   
8  7adf8ddd-3ad8-4dc3-a55b-395beb8aac5e, bf257382...   

                                    timestampHistory  \
3  1658333312180, 1658404553818, 1658408449062, 1...   
4  1658766608801, 1658766608801, 1660084035094, 1...   
5  16582

teste para confirmar a filtragem

In [ ]:
# Verificar se todos os userIds filtrados realmente pertencem ao dataset de validação
print("Todos os IDs do dataset filtrado estão no dataset de validação?", df_filtrado["userId"]
      .isin(user_ids_validacao).all())

# Mostrar a contagem de IDs no dataset filtrado
print("Número de registros antes do filtro:", len(df_final_treino))
print("Número de registros após o filtro:", len(df_filtrado))


Todos os IDs do dataset filtrado estão no dataset de validação? True
Número de registros antes do filtro: 577942
Número de registros após o filtro: 112184


Salvar o dataset cruzado

In [15]:
df_filtrado.to_csv("../data/filtered/filtered_dataset.csv", index=False)

print("\nDataset atualizado salvo ")



Dataset atualizado salvo 


In [16]:
df_filtrado.head()


,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
3,c1e8d644329a78ea1f994292db624c57980b2886cfbc2d...,Non-Logged,56,"1f2b9c2f-a2d2-4192-b009-09065da8ec23, 04756569...","1658333312180, 1658404553818, 1658408449062, 1...","8, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 1, 1...","182696, 91925, 30000, 273655, 126409, 42980, 1...","58.26, 72.66, 22.57, 59.89, 40.36, 36.35, 14.7...","1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1...","1658333312180, 1658404553818, 1658408449062, 1..."
4,e777d1f31d4d955b63d60acc13df336d3903f52ab8f8f4...,Non-Logged,4,"bebdeb3e-1699-43e0-a1b8-989f5a6ab679, f4b484a7...","1658766608801, 1658766608801, 1660084035094, 1...","579, 579, 7, 2","801396, 801396, 10000, 10000","78.74, 78.74, 16.71, 9.34","7, 7, 1, 1","1658766608801, 1658766608801, 1660084035094, 1..."
5,6aa5109374f6534b0400ccbbdb9c7b7d64c40a663fe51c...,Non-Logged,4,"7fe849c0-4a55-429d-b480-11ee216909dd, 2de36771...","1658259718982, 1658424852871, 1658859110243, 1...","127, 155, 6, 78","350199, 320000, 39671, 151452","20.83, 80.94, 45.18, 22.32","12, 2, 1, 4","1658259718982, 1658424852871, 1658859110243, 1..."
7,c33bcbcaf32fe895fb0a854c94531120657ad5adabe3e1...,Non-Logged,3,"0b8f3728-4ef7-40e7-85ac-367d8039bb6d, c8ade35b...","1657911681110, 1658159870839, 1660136510555","1, 0, 0","40000, 30000, 36068","10.07, 25.76, 37.78","1, 1, 2","1657911681110, 1658159870839, 1660136510555"
8,3c23b4ee53e663cb2d47eed9b5e7216cb7e6784e7f7722...,Non-Logged,4,"7adf8ddd-3ad8-4dc3-a55b-395beb8aac5e, bf257382...","1656724891913, 1657221969625, 1659703496742, 1...","0, 4, 1, 1","1219245, 150684, 58525, 12194","19.24, 51.29, 43.07, 43.27","67, 1, 1, 1","1656724891913, 1657221969625, 1659703496742, 1..."


Convertando o history em um lista de UUIDs individuais

In [17]:
df_filtrado["history"] = df_filtrado["history"].apply(lambda x: str(x).split(", "))


C:\Users\miche\AppData\Local\Temp\ipykernel_15012\2249611568.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["history"] = df_filtrado["history"].apply(lambda x: str(x).split(", "))


In [19]:
df_filtrado.head()



,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
3,c1e8d644329a78ea1f994292db624c57980b2886cfbc2d...,Non-Logged,56,"[1f2b9c2f-a2d2-4192-b009-09065da8ec23, 0475656...","1658333312180, 1658404553818, 1658408449062, 1...","8, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 1, 1...","182696, 91925, 30000, 273655, 126409, 42980, 1...","58.26, 72.66, 22.57, 59.89, 40.36, 36.35, 14.7...","1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1...","1658333312180, 1658404553818, 1658408449062, 1..."
4,e777d1f31d4d955b63d60acc13df336d3903f52ab8f8f4...,Non-Logged,4,"[bebdeb3e-1699-43e0-a1b8-989f5a6ab679, f4b484a...","1658766608801, 1658766608801, 1660084035094, 1...","579, 579, 7, 2","801396, 801396, 10000, 10000","78.74, 78.74, 16.71, 9.34","7, 7, 1, 1","1658766608801, 1658766608801, 1660084035094, 1..."
5,6aa5109374f6534b0400ccbbdb9c7b7d64c40a663fe51c...,Non-Logged,4,"[7fe849c0-4a55-429d-b480-11ee216909dd, 2de3677...","1658259718982, 1658424852871, 1658859110243, 1...","127, 155, 6, 78","350199, 320000, 39671, 151452","20.83, 80.94, 45.18, 22.32","12, 2, 1, 4","1658259718982, 1658424852871, 1658859110243, 1..."
7,c33bcbcaf32fe895fb0a854c94531120657ad5adabe3e1...,Non-Logged,3,"[0b8f3728-4ef7-40e7-85ac-367d8039bb6d, c8ade35...","1657911681110, 1658159870839, 1660136510555","1, 0, 0","40000, 30000, 36068","10.07, 25.76, 37.78","1, 1, 2","1657911681110, 1658159870839, 1660136510555"
8,3c23b4ee53e663cb2d47eed9b5e7216cb7e6784e7f7722...,Non-Logged,4,"[7adf8ddd-3ad8-4dc3-a55b-395beb8aac5e, bf25738...","1656724891913, 1657221969625, 1659703496742, 1...","0, 4, 1, 1","1219245, 150684, 58525, 12194","19.24, 51.29, 43.07, 43.27","67, 1, 1, 1","1656724891913, 1657221969625, 1659703496742, 1..."


In [85]:
print(df_itens.columns)


Index(['page', 'url', 'issued', 'title', 'body', 'caption', 'title_tfidf',
       'body_tfidf', 'caption_tfidf'],
      dtype='object')


In [87]:

# Garantir que history e timestampHistory sejam listas sem modificar os valores
df_filtrado["history"] = df_filtrado["history"].apply(lambda x: x.split(", ") if isinstance(x, str) else x)

# Explodir apenas 'history' para permitir a junção com df_itens
df_exploded = df_filtrado.explode("history")

# Fazer o merge para encontrar as páginas correspondentes
df_merged = df_exploded.merge(df_itens, left_on="history", right_on="page", how="inner")

# Voltar a agrupar os dados, mantendo timestampHistory e timestampHistory_new como listas originais
df_final = df_merged.groupby(["userId", "history"]) \
    .agg({
        "timestampHistory": "first",  # Mantém a lista original de timestamps
        "timestampHistory_new": "first",  # Mantém a lista original convertida
        "page": "first",
        "url": "first",
        "issued": "first",
        "title": "first",
        "body": "first",
        "caption": "first"
    }).reset_index()



C:\Users\miche\AppData\Local\Temp\ipykernel_15012\2906795242.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["history"] = df_filtrado["history"].apply(lambda x: x.split(", ") if isinstance(x, str) else x)


In [88]:
df_final.head()

,userId,history,timestampHistory,timestampHistory_new,page,url,issued,title,body,caption
0,00025217b55f104ede326550e7b46c3969a5a5333fcd29...,28965b9d-24c7-45d7-9922-0f965e47e92b,[1658342522783],1658342522783,28965b9d-24c7-45d7-9922-0f965e47e92b,http://g1.globo.com/rj/rio-de-janeiro/noticia/...,2022-07-19 19:24:30+00:00,prender homem suspeitar lançar explosivo consu...,vídeo divulgar época crime mostrar ataque cons...,vídeo câmerar segurança mostrar ataque setembr...
1,0003f534fe8124d98eef78b01baa3b69e3ffcda6e5a657...,07fc1034-4aa5-4600-b19d-5160c55c917d,"[1656844828006, 1657017605361, 1657017766954, ...","1656844828006, 1657017605361, 1657017766954, 1...",07fc1034-4aa5-4600-b19d-5160c55c917d,http://g1.globo.com/ce/ceara/noticia/2022/07/0...,2022-07-04 20:46:19+00:00,cearense viralizar bordão papai homem morrer d...,autor bordão papai morrer cear cearense iatami...,iatami bezerrar ano iaiá falecer domingo passa...
2,0003f534fe8124d98eef78b01baa3b69e3ffcda6e5a657...,2d09c185-98d3-4898-a76c-12ba58eabd04,"[1656844828006, 1657017605361, 1657017766954, ...","1656844828006, 1657017605361, 1657017766954, 1...",2d09c185-98d3-4898-a76c-12ba58eabd04,http://g1.globo.com/sp/sao-paulo/eleicoes/2022...,2022-08-04 00:27:47+00:00,impasse tse garantir candidatura presidência p...,durante coletivo imprensa reprodução youtube d...,pro disputar interno comando siglar stj devolv...
3,0003f534fe8124d98eef78b01baa3b69e3ffcda6e5a657...,855d20b7-53f2-4678-a10f-55402d085018,"[1656844828006, 1657017605361, 1657017766954, ...","1656844828006, 1657017605361, 1657017766954, 1...",855d20b7-53f2-4678-a10f-55402d085018,http://g1.globo.com/go/goias/noticia/2022/07/0...,2022-07-04 08:54:20+00:00,filho caiar encontrar matar fazenda crixás,caiar filho filho governador goiás divulgaçãog...,filho governador formar administração causo mo...
4,0003f534fe8124d98eef78b01baa3b69e3ffcda6e5a657...,900838d8-abf5-4cfb-9308-87a565f7c51f,"[1656844828006, 1657017605361, 1657017766954, ...","1656844828006, 1657017605361, 1657017766954, 1...",900838d8-abf5-4cfb-9308-87a565f7c51f,http://g1.globo.com/ce/ceara/noticia/2022/07/0...,2022-07-02 08:04:12+00:00,conhecer arrar céu alto fortaleza quase dobro ...,andar metro altura edifício one alto fortaleza...,one apartamento residencia milhão milhão ver m...


In [ ]:
df_resultado.to_csv("../data/preprocessed/itens/matching_history_pages/merged_history_pages.csv", index=False, encoding="utf-8")
